# Question 1

$$
\frac{dy(x)}{dx} = y'(x) = -50(y - \cos(x))
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from true import exact_solution, compute_error

from solvers.euler_explicit import explicit_euler
from solvers.euler_implicit import implicit_euler
from solvers.midpoint import midpoint
from solvers.trapezoidal import trapezoidal
from solvers.ab_2 import adams_bashforth2
from solvers.rk_2 import rk2
from solvers.rk_4 import rk4

# Define the ODE
def f(x, y):
    return -50 * (y - np.cos(x))

# Parameters
y0 = 0
x_range = [0, 1]

# Get exact solution
sol_exact = exact_solution()
x_exact = np.linspace(0, 1, 1000)
y_exact = sol_exact.sol(x_exact)[0]

# Define step sizes to test
step_sizes = [0.01, 0.005, 0.002, 0.001]

# Dictionary of methods
methods = {
    'Explicit Euler': explicit_euler,
    'Implicit Euler': implicit_euler,
    'Midpoint': midpoint,
    'Trapezoidal': trapezoidal,
    'Adams-Bashforth2': adams_bashforth2,
    'RK2 (Heun)': rk2,
    'RK4': rk4
}

# Plot each method separately with different step sizes
for method_name, method_func in methods.items():
    plt.figure(figsize=(10, 6))
    
    # Plot exact solution
    plt.plot(x_exact, y_exact, 'k-', linewidth=2, label='Exact (MATLAB-like)')
    
    # Plot numerical solutions for different step sizes
    for i, h in enumerate(step_sizes):
        x_num, y_num = method_func(f, y0, x_range, h)
        
        # Interpolate exact solution to numerical grid for error calculation
        y_exact_interp = sol_exact.sol(x_num)[0]
        error = compute_error(y_num, y_exact_interp)
        
        plt.plot(x_num, y_num, '--', linewidth=1.5, 
                label=f'h={h}, error={error:.2e}')
    
    plt.xlabel('x')
    plt.ylabel('y(x)')
    plt.title(f'{method_name} Method - ODE: dy/dx = -50(y - cos(x))')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{method_name.replace(" ", "_")}_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Print errors table
    print(f"\n{method_name} Errors:")
    print("-" * 40)
    print("Step size h | Relative L2 Error")
    print("-" * 40)
    for h in step_sizes:
        x_num, y_num = method_func(f, y0, x_range, h)
        y_exact_interp = sol_exact.sol(x_num)[0]
        error = compute_error(y_num, y_exact_interp)
        print(f"    {h:6.4f}   |   {error:.4e}")
    print()

# Convergence study plot
plt.figure(figsize=(10, 6))
for method_name, method_func in methods.items():
    errors = []
    for h in step_sizes:
        x_num, y_num = method_func(f, y0, x_range, h)
        y_exact_interp = sol_exact.sol(x_num)[0]
        error = compute_error(y_num, y_exact_interp)
        errors.append(error)
    
    plt.loglog(step_sizes, errors, 'o-', label=method_name)

plt.xlabel('Step size h')
plt.ylabel('Relative L2 Error')
plt.title('Convergence of ODE Solvers')
plt.legend()
plt.grid(True, which="both", ls="-", alpha=0.3)
plt.tight_layout()
plt.savefig('convergence_study.png', dpi=150, bbox_inches='tight')
plt.show()

: 